# Basic setup, exploration and Zero Shot inference
### Model Used: FLAN T5,

In [4]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Imports
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [6]:
# Data Loading

huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

# Check to see what the data looks like:

example_indices = [10, 20]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/anshriyal/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
BASEL

In [7]:
# Loading Model:

# 1. Google FLAN T5

model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

### Observing the results of the base model without prompt engineering
### Also, experimenting with multiple prompts all with 0 shot inference
<br>
The point of experimenting with multiple prompts is to see the effect (or lack thereof) when adding more information to yout prompts.<br>
This helps determine whether more complex and costly models are even required for your use case.

### Dialogue 1:
As we have observed above, the structure of the base input is simply a conversation between individuals.<br>
Without providing any additional prefix prompt (eg. Summarize the following:) or suffix prompt (eg. Summarize the above conversation) or more complicated combinations (eg. This is a conversation, remember the important things {here comes the conversation} now use the important parts to summarize the conversation)
<br><br>
For this dialogue, we will send the conversation directly to the model, hoping that it will understand what we want it to do.<br>
final dialogue will be the same as the base dialogue.

### Dialogue 2:
Here, we will add a prefix prompt "Summarize the following conversation between individuals:"<br>
We will discuss all the results after seeing some of the summaries.

### Dialogue 3:
Here, we will add a suffix prompt "summarize what you heard"

### Dialogue 4:
Here, we will add both the previous prefix suffix prompts

### Dialogue 5:
This is here just to show how big of an impact zero shot learning can cause if used carelessly.<br>
Here, we will work with the prefix prompt "Here is some text" and the suffix prompt "what did it say?" and compare the difference such a small lack of clarity can cause<br>
To exemplify the risk to the process without the adequate attention to detail required, dialogue 5 will not use any formatting in the prefix and suffix prompts

In [9]:
for i, index in enumerate(example_indices):
    summary = dataset['test'][index]['summary']
    
    base_dialogue = dataset['test'][index]['dialogue']
    
    # First Dialogue has no prefix or suffix
    dialogue_1 = "" + base_dialogue + ""
    
    # Second Dialogue has just a prefix
    dialogue_2 = "Summarize the following conversation between individuals:\n" + base_dialogue + ""
    
    # Third Dialogue has just a suffix
    dialogue_3 = "" + base_dialogue + "\n Summarize what you heard"
    
    # Fourth Dialogue has both a prefix and a suffix
    dialogue_4 = "Summarize the following conversation between individuals:\n" + base_dialogue + "\n Summarize what you heard"
    
    # Fifth Dialogue has both a prefix and a suffix
    dialogue_5 = "Here is some text" + base_dialogue + "what did it say?"
    
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{base_dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)


    # Dialogue 1:
    inputs = tokenizer(dialogue_1, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')


    # Dialogue 2:
    inputs = tokenizer(dialogue_2, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    print(f'Zero Shot (Prefix Only):\n{output}\n')


    # Dialogue 3:
    inputs = tokenizer(dialogue_3, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    print(f'Zero Shot (Suffix Only):\n{output}\n')



    # Dialogue 4:
    inputs = tokenizer(dialogue_4, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    print(f'Zero Shot (Both Prefix and Suffix):\n{output}\n')


    # Dialogue 5:
    inputs = tokenizer(dialogue_5, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    print(f'Zero Shot (Bad version):\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
BASELIN

In [ ]:
# Loading Model:

# 2. Llama 2 7B

model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Discussion:
With Zero Shot, it is important not to expect perfect results. Base models such as FLAN T5 and Llama are trained to continue conversations as humans do, not specifically to summarize anything.<br>
More often than not, the model needs some kind of one shot or few shot prompt engineering to be able to understand what we want it to do.